In [31]:
import pandas as pd
import numpy as np

In [32]:
data = pd.read_csv('data/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [33]:
print(data.columns)
print(data.shape)
print(data.isnull().sum())

Index(['review', 'sentiment'], dtype='object')
(50000, 2)
review       0
sentiment    0
dtype: int64


In [34]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [64]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
#This code initializes the variable 'stopwords' with a list of English stopwords.

stopwords = nltk.corpus.stopwords.words('english')

In [47]:
def clean_text(text):
    text = re.sub(r'<.*?>', '', text) #Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text) #Remove punctuation
    text = text.lower() #Convert to lowercase
    text = text.split() #Split the text
    text = [word for word in text if not word in set(stopwords)] #Remove stopwords
    text = ' '.join(text)
    return text



In [39]:
data['review'] = data['review'].apply(clean_text)


In [40]:
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


In [44]:
def stemmer(text):
    text = text.split()
    stemmer = nltk.PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = ' '.join(stemmed_words)
    return text

In [45]:
data['review'] = data['review'].apply(stemmer)

In [46]:
data.head()

,review,sentiment
0,one review mention watch oz episod hook right ...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,positive


Train and Test Data

In [51]:
train_reviews = data.review[:30000]
test_reviews = data.review[30000:]


BAG of WORDS

In [57]:
cv = CountVectorizer(max_df=1, binary=False, ngram_range=(1,3))

cv_train_reviews = cv.fit_transform(train_reviews)
cv_test_reviews = cv.transform(test_reviews)

print('BOW_cv_train:', cv_train_reviews.shape)
print('BOW_cv_test:', cv_test_reviews.shape)

BOW_cv_train: (30000, 4483092)
BOW_cv_test: (20000, 4483092)


TF-IDF

In [56]:
tf = TfidfVectorizer(min_df=0.0, max_df=1, use_idf=True, ngram_range=(1,3))

tf_train_reviews = tf.fit_transform(train_reviews)
tf_test_reviews = tf.transform(test_reviews)

print('Tfidf_train:', tf_train_reviews.shape)
print('Tfidf_test:', tf_test_reviews.shape)

Tfidf_train: (30000, 4483092)
Tfidf_test: (20000, 4483092)


Label Encoding : 
1 coresponds to positive sentiment
0 coresponds to negative sentiment

In [61]:
from sklearn.preprocessing import LabelBinarizer
label = LabelBinarizer()
sentiment_data = label.fit_transform(data['sentiment'])
# sentiment_data

In [62]:
label_train_data = sentiment_data[:30000]
label_test_data = sentiment_data[30000:]

We use SVM to classify the sentiment of the review in bow and tf-idf

In [66]:
svm_model = SVC()
svm_bow = svm_model.fit(cv_train_reviews, label_train_data) #BOW
svm_tfidf = svm_model.fit(tf_train_reviews, label_train_data) #Tfidf

/home/lahad/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# Predict the model for BOW 
svm_bow_predict = svm_model.predict(cv_test_reviews)
svm_bow_score = classification_report(label_test_data, svm_bow_predict)
print("SVM BOW:", svm_bow_score)


In [ ]:
# Predict the model for Tfidf
svm_tfidf_predict = svm_model.predict(tf_test_reviews)
svm_tfidf_score = classification_report(label_test_data, svm_tfidf_predict)
print("SVM Tfidf:", svm_tfidf_score)